In [ ]:
from numpy import arange, array, exp, finfo, float64, hypot, max, ones, pad, pi, round, sum, tan, where, zeros_like, zeros
from matplotlib.image import imread
from matplotlib.pyplot import axis, figure, imshow, imsave, subplot, show, title, tight_layout, savefig
from scipy.ndimage import label
% matplotlib inline

"""
Naive implementation of Canny edge detector.
Reference:
https://en.wikipedia.org/wiki/Canny_edge_detector.
"""

def luminance_convert(image):
    """
    Convert RGB color to luminance.
    Formula: Luminance = (0.2126*R + 0.7152*G + 0.0722*B) 

    Input:
    - image: Input array of shape (H, W, 3)
    
    Returns:
    - out: Output data, of shape (H, W)          
    """
    H, W, _ = image.shape
    out = zeros((H,W))
    out = 0.2126*image[:,:,0]+0.7152*image[:,:,1]+0.0722*image[:,:,2]
    out = round(out, 2)
    return out

def gaussian_filter(image, sigma = 1):
    """
    A naive implementation of gaussian filter on image luminance.

    Input:
    - image: Input data of shape (H, W)
    - sigma: standard deviation of Gaussian filter.

    Returns: 
    - out: Output data, of shape (H, W) 
    """
    # Calculate 1d Gaussian filter coefficient.
    halfwidth = 3*sigma
    x = arange(-halfwidth, halfwidth+1, 1)
    gau_1d = exp(-1/(2*sigma**2)*(x**2))
    gau_1d = round(gau_1d, 2)
    gau_1d = gau_1d/sum(gau_1d)

    H,W = image.shape
    WW = len(gau_1d)
    out = zeros_like(image)
    p = int((WW-1)/2)

    # Row convolution of 1d gaussian filter
    img_pad = pad(image, ((0,0), (p, p)), 'edge')
    
    for r in range(0, H):
        for c in range(0, W):
            out[r, c] = sum(img_pad[r, c:c+WW]*gau_1d)    

    # Column convolution of 1d gaussian filter
    img_pad = pad(out, ((p,p), (0, 0)), 'edge')
    
    for c in range(0, W):
        for r in range(0, H):
            out[r, c] = sum(img_pad[r:r+WW, c]*gau_1d)
    
    out = round(out, 2)
    return out

def Sobel_filter(image):
    """
    A naive implementation of Sobel filter on input image.

    Input:
    - image: Input data of shape (H, W)

    Returns: 
    - out_x: Sobel filter on x-axis, output data of shape (H, W) 
    - out_y: Sobel filter on y-axis, output data of shape (H, W)     
    """

    Gx = array([[1,0,-1],[2,0,-2],[1,0,-1]])
    Gy = Gx.T
    H, W = image.shape

    img_pad = pad(image, ((1,1), (1, 1)), 'edge')
    sobel_x = zeros_like(image)
    sobel_y = zeros_like(image)

    # Convolution of Gx filter.
    for r in range(0, H):
        for c in range(0, W):
            sobel_x[r, c] = sum(img_pad[r:r+3, c:c+3]*Gx)

    # Convolution of Gy filter.
    for r in range(0, H):
        for c in range(0, W):
            sobel_y[r, c] = sum(img_pad[r:r+3, c:c+3]*Gy)
    
    return sobel_x, sobel_y

def Non_max_suppression(sobel_x, sobel_y):
    """
    A naive implementation of Non-maximum suppression.

    Input:
    - sobel_x: gradient at x-axis after Sobel operator, input data of shape (H, W).
    - sobel_y: gradient at y-axis after Sobel operator, input data of shape (H, W).
    
    Returns:
    - edge: edge pixel with gradient magnitude, output data of shape (H, W).
    - mask: mask for edge, output data of shape (H, W), dtype=bool
    """    
    magnitude = hypot(sobel_x, sobel_y)
    
    # Replace zero value with minimum value to calculate tan value
    sobel_x = where(sobel_x==0, finfo(float64).eps, sobel_x)
    angle = sobel_y/sobel_x
    
    # Categorizes the continuous gradient directions into a discrete directions of 
    # 0 degree, 45 degree, 90 degree, 135 degree
    mask_0 = (angle<=tan(pi/8)) & (angle>=tan(7*pi/8))
    mask_45 = (angle<=tan(3*pi/8)) & (angle>=tan(pi/8))
    mask_90 = (angle<=tan(5*pi/8)) | (angle>=tan(3*pi/8))
    mask_135 = (angle<=tan(7*pi/8)) & (angle>=tan(5*pi/8)) 
    
    # Compare edge strength of the current pixel to the other pixels in the mask with the same direction, 
    # preserve current edge strength only when it is largest in comparison.
    # ----- 0 degrees ------
    mag_l = pad(magnitude, ((0,0),(0,1)), 'constant', constant_values = 0)
    mag_l = mag_l[:,1:]
    left = (magnitude*mask_0)>(mag_l*mask_0)

    mag_r = pad(magnitude, ((0,0),(1,0)), 'constant', constant_values = 0)
    mag_r = mag_r[:,:-1]
    right = (magnitude*mask_0)>(mag_r*mask_0)

    mask_0_1 = left & right
    
    # ----- 90 degrees ------
    mag_t = pad(magnitude, ((1,0),(0,0)), 'constant', constant_values = 0)
    mag_t = mag_t[:-1, :]
    top = (magnitude*mask_90)>(mag_t*mask_90)

    mag_b = pad(magnitude, ((0,1),(0,0)), 'constant', constant_values = 0)
    mag_b = mag_b[1:, :]

    bottom = (magnitude*mask_90)>(mag_b*mask_90)
    mask_90_1 = top & bottom
    
    # ----- 45 degrees ------
    mag_ne = pad(magnitude, ((1,0),(0,1)), 'constant', constant_values = 0)
    mag_ne = mag_ne[:-1, 1:]
    ne = (magnitude*mask_45)>(mag_ne*mask_45)

    mag_sw = pad(magnitude, ((0,1),(1,0)), 'constant', constant_values = 0)
    mag_sw = mag_sw[1:, :-1]
    sw = (magnitude*mask_45)>(mag_sw*mask_45)

    mask_45_1 = ne & sw
    
    # ----- 135 degrees ------
    mag_nw = pad(magnitude, ((1,0),(1,0)), 'constant', constant_values = 0)
    mag_nw = mag_nw[:-1, :-1]
    nw = (magnitude*mask_135)>(mag_nw*mask_135)

    mag_se = pad(magnitude, ((0,1),(0,1)), 'constant', constant_values = 0)
    mag_se = mag_se[1:, 1:]
    se = (magnitude*mask_135)>(mag_se*mask_135)
    mask_135_1 = nw & se
    
    mask = (mask_0_1)|(mask_90_1)|(mask_45_1)|(mask_135_1)
    edge = magnitude*mask
    
    return edge, mask, magnitude   

def hysteresis(edge, mask, low_threshold=None, high_threshold=None):
    """
    Double thresholding. 
    Preserve weak edge pixels connected to strong edge pixels as good edge pixels.

    Input:
    - edge: edge pixel with gradient magnitude, input data of shape (H, W).
    - mask: mask for edge, input data of shape (H, W), dtype=bool
    - low_threshold : float
        Lower bound for hysteresis thresholding (linking edges).
        If None, low_threshold is set to 10% of max gradient magitude.
    - high_threshold : float
        Upper bound for hysteresis thresholding (linking edges).
        If None, high_threshold is set to 20% of max gradient magitude.

    Returns:
    - good_edge: binary edge map, output data of shape (H, W).
    """

    # Create two masks at the two thresholds.
    if low_threshold is None:
        low_threshold = 0.1 * max(edge)

    if high_threshold is None:
        high_threshold = 0.2 * max(edge)

    high_mask = mask & (edge >= high_threshold)
    low_mask = mask & (edge >= low_threshold)

    # Label connected pixels in the low threshold mask. If labeled pixels in low threshold
    # mask is also positive in high threshold mask, preserve labeled pixels as good edge.
    strel = ones((3, 3), bool)
    labels, count = label(low_mask, strel)

    good_edge = zeros_like(mask)

    for i in arange(1, count+1):
        if sum(high_mask[labels==i])>0:
            good_edge[labels==i]=1

    return good_edge

In [ ]:
def image_show(image, figsize=(10,8), cmap=None):
    figure(figsize = (10,8))
    if cmap:
        imshow(image, cmap=cmap)
    else:
        imshow(image)
    axis('off')
    tight_layout()
    return figure

In [ ]:
image = imread('bird.jpg')
image_show(image)

In [ ]:
luminance = luminance_convert(image)
image_show(luminance)

In [ ]:
gaussian = gaussian_filter(luminance)
image_show(gaussian)

In [ ]:
sobel_x, sobel_y = Sobel_filter(gaussian)

In [ ]:
image_show(sobel_x)

In [ ]:
image_show(sobel_y)

In [ ]:
edge, mask, magnitude = Non_max_suppression(sobel_x, sobel_y)

In [ ]:
image_show(magnitude, cmap='Greys')

In [ ]:
image_show(edge, cmap='Greys')

In [ ]:
canny_edge = hysteresis(edge, mask)

In [ ]:
image_show(canny_edge, cmap='Greys')

In [ ]:
canny_edge2 = hysteresis(edge, mask, 10, 30)

In [ ]:
image_show(canny_edge2, cmap='Greys')

In [ ]:
figure(figsize=(20,5))
subplot(1,3,1)
title("Original image")
imshow(image)
axis('off')

subplot(1,3,2)
title("Luminance")
imshow(luminance)
axis('off')

subplot(1,3,3)
title("Gaussian blur")
imshow(gaussian)
axis('off')

tight_layout()

show()

In [ ]:
figure(figsize=(14,5))
subplot(1,2,1)
title("Gradient at x axis")
imshow(sobel_x)
axis('off')

subplot(1,2,2)
title("Gradient at y axis")
imshow(sobel_y)
axis('off')

tight_layout()
show()

In [ ]:
figure(figsize=(14,5))
subplot(1,2,1)
title("Gradient magnitude before Non-maximum suppression")
imshow(magnitude, cmap = 'Greys')
axis('off')

subplot(1,2,2)
title("Gradient magnitude after Non-maximum suppression")
imshow(edge, cmap = 'Greys')
axis('off')

tight_layout()
show()

In [ ]:
figure(figsize=(14,5))
subplot(1,2,1)
title("Default double threshold")
imshow(canny_edge, cmap = 'Greys')
axis('off')

subplot(1,2,2)
title("Low threshold = 10, High threshold = 30")
imshow(canny_edge2, cmap = 'Greys')
axis('off')

tight_layout()
show()